# Segmentation visualization demo

In [1]:
# Import libraries
import os
import napari
## note: using napari with a remote server might cause errors
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

from tifffile import imread
from skimage.measure import label, regionprops

In [2]:
# Set up the paths
base_path = 'Z:/Data/Analyzed/sample-dataset-output/cell-culture-3D'
signal_path = os.path.join(base_path, 'signal')
expr_path = os.path.join(base_path, 'expr')
image_path = os.path.join(base_path, 'images')
figure_path = os.path.join(base_path, 'figures')
if not os.path.exists(figure_path):
    os.mkdir(figure_path)

# Load the data 
adata = sc.read_h5ad(os.path.join(expr_path, '2023-12-08-cell_culture.h5ad'))
adata

AnnData object with n_obs × n_vars = 1950 × 998
    obs: 'area', 'x', 'y', 'seg_label'
    layers: 'cytoplasm', 'nucleus'

In [3]:
# Load images (using the scaled down version for better performance)
cell = imread(os.path.join(image_path, 'fused', 'Cell_05.tif'))
nuclei = imread(os.path.join(image_path, 'fused', 'Nuclei_05.tif'))

In [4]:
# Load reads dataframe 
reads_df = pd.read_csv(os.path.join(signal_path, 'fused_goodSpots.csv'))
reads_df['x_scaled'] = (reads_df['x'] - 1) * .5
reads_df['y_scaled'] = (reads_df['y'] - 1) * .5
reads_df['z'] = reads_df['z'] - 1

reads_df['x_scaled'] = reads_df['x_scaled'].astype(np.int32)
reads_df['y_scaled'] = reads_df['y_scaled'].astype(np.int32)

reads_df['reads_assignment'] = cell[reads_df['z'].values,  reads_df['y_scaled'].values, reads_df['x_scaled'].values]
reads_df

,x,y,z,gene,x_scaled,y_scaled,reads_assignment
0,374,1197,12,ABL1,186,598,0
1,490,1146,12,TCF19,244,572,0
2,624,1178,12,ZFP36L2,311,588,0
3,652,845,12,AXIN1,325,422,0
4,677,1189,12,MYL9,338,594,0
...,...,...,...,...,...,...,...
2076347,8885,12694,32,NCAPD2,4442,6346,0
2076348,9102,13170,32,TCF3,4550,6584,0
2076349,9314,13566,32,ADNP2,4656,6782,0
2076350,9538,12709,32,UNC50,4768,6354,0


In [5]:
# Measure regional properties
cell_props = regionprops(cell, nuclei)

In [6]:
# View selected cells
cell_seg_label = 106

In [7]:
# Get reads for selected cell
selected_reads = reads_df.loc[reads_df['reads_assignment'] == cell_seg_label, :].copy()
selected_reads['z'] = selected_reads['z'] - cell_props[cell_seg_label-1].bbox[0]
selected_reads['y_scaled'] = selected_reads['y_scaled'] - cell_props[cell_seg_label-1].bbox[1]
selected_reads['x_scaled'] = selected_reads['x_scaled'] - cell_props[cell_seg_label-1].bbox[2]
selected_reads

,x,y,z,gene,x_scaled,y_scaled,reads_assignment
170887,7805,731,0,EIF3B,5,65,106
170889,7807,729,0,EIF3B,6,64,106
170893,7814,710,0,TSHZ1,9,54,106
170899,7824,717,0,NFIC,14,58,106
170900,7824,731,0,MAZ,14,65,106
...,...,...,...,...,...,...,...
204461,7927,751,16,RPL37,66,75,106
204556,7883,727,17,SRSF7,44,63,106
204559,7922,730,17,INTS5,63,64,106
204650,7915,755,18,TOR1B,60,77,106


In [8]:
# Napari visualization
viewer = napari.Viewer()
cell_layer = viewer.add_image(cell_props[cell_seg_label-1].image, name='cell', scale=(2, 1, 1), colormap='green')
cell_layer.bounding_box.visible = True
nuclei_layer = viewer.add_image(cell_props[cell_seg_label-1].intensity_image, name='nuclei', scale=(2, 1, 1), colormap='blue')
reads_layer = viewer.add_points(selected_reads.loc[:, ['z', 'y_scaled', 'x_scaled']].values, scale=(2, 1, 1), size=2)
viewer.dims.ndisplay = 3

Here is an example of the napari visualization
![napari_example](./napari_example.png)